# 🤖 Multilayer Perceptron

Esta Jupyter Notebook implementa una MLP para detectar patrones en una matriz 10x10.
Se genera un dataset para luego entrenar el modelo y validarlo.
Se evalúa la precisión para reiterar sobre el modelo con el objetivo de mejorar su eficacia.
Finalmente se exporta el modelo para ser utilizado en la aplicación web.


## 📋 Situación

En una matriz 10x10 se debe poder detectar las letras `b`, `d`, `f` que se corresponden a los siguientes patrones:

![Patrones que el MLP deberá reconocer](assets/patterns.png)

Ante un dato nuevo, el MLP deberá ser capaz de clasificar el contenido de esa matriz en uno de los tres patrones.


In [1]:
import pandas as pd
import numpy as np
from typing import Literal, List

# Extend maximum width when printing DataFrames so they fit in just one line
pd.options.display.width = 100

# Set seed for reproducibility
SEED = 7
RNG = np.random.default_rng(SEED)

# fmt: off
# Define the 10x10 patterns for 'b', 'd', and 'f' as a 1D array
PATTERNS = {
    "b": np.array(
           [
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
            0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
            0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
            0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
            0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        ],
        dtype=np.uint8,
    ),
    "d": np.array(
        [
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
            0, 0, 0, 1, 1, 1, 1, 1, 0, 0,
            0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
            0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
            0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
            0, 0, 0, 1, 1, 1, 1, 1, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        ],
        dtype=np.uint8,
    ),
    "f": np.array(
        [
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
            0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
            0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
            0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
            0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        ],
        dtype=np.uint8,
    ),
}
# fmt: on

## 🗃️ Generación de Datasets

Conviene generar el conjunto de datos de manera programática.
Los datasets deberán ser representativos a la hora de definir la distribución de los ejemplos de entrenamiento.
Se definen los patrones 'b', 'd', y 'f'.
Luego, se crea una función `generate_sample` que crea un ejemplo de un patrón dado con una distorsión entre 0 y 0.3. 


In [2]:
def generate_sample(pattern: Literal["b", "d", "f"], noise: float = 0.0) -> np.ndarray:
    """
    Generates a 1D array based on a given pattern letter ('b', 'd', or 'f'), with optional noise.

    Args:
        pattern: One of 'b', 'd', or 'f'.
        noise: Proportion of pixels to flip.
    Returns:
        A numpy 1D array representing the 10x10 matrix with the pattern and noise applied.
    """
    sample = PATTERNS[pattern].copy()
    num_pixels = sample.size
    num_noisy = int(noise * num_pixels)

    if num_noisy > 0:
        # Choose random indices to flip
        indices = RNG.choice(num_pixels, num_noisy, replace=False)
        # Flip the selected pixels (0 becomes 1, 1 becomes 0)
        sample[indices] = 1 - sample[indices]

    return sample


def print_sample(sample: np.ndarray):
    """
    Pretty-prints the provided 1D array as a 10x10 matrix,
    coloring filled pixels bold green and adding an outline to the matrix.

    Args:
        sample: 1D numpy array of 0s and 1s.
    """
    BOLD_GREEN = "\033[1;92m"
    RESET = "\033[0m"
    for i in range(10):
        row = sample[i * 10 : (i + 1) * 10]
        print(" ".join([f"{BOLD_GREEN}1{RESET}" if val else "0" for val in row]))


print("A 'b' pattern with zero noise:")
print_sample(generate_sample("d", 0.00))
print()
print("A 'b' with 30% noise, meaning 30 out of 100 cells have been randomly flipped:")
print_sample(generate_sample("d", 0.30))

A 'b' pattern with zero noise:
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 1 0 0
0 0 0 0 0 0 0 1 0 0
0 0 0 0 0 0 0 1 0 0
0 0 0 1 1 1 1 1 0 0
0 0 1 0 0 0 0 1 0 0
0 0 1 0 0 0 0 1 0 0
0 0 1 0 0 0 0 1 0 0
0 0 0 1 1 1 1 1 0 0
0 0 0 0 0 0 0 0 0 0

A 'b' with 30% noise, meaning 30 out of 100 cells have been randomly flipped:
1 0 0 0 1 0 0 0 0 0
1 1 0 0 0 0 0 0 0 0
0 0 0 1 1 0 1 0 0 0
0 1 0 0 0 0 0 1 0 0
0 0 1 0 1 0 1 0 0 1
1 0 1 0 0 0 0 1 1 0
0 0 1 0 1 0 1 0 1 0
1 1 1 0 1 1 0 1 0 0
0 0 0 1 1 1 1 0 0 1
0 0 0 0 0 1 1 1 0 0


Luego se generan 3 datasets que contengan 100, 500 y 1000 ejemplos. 
El 10% serán patrones sin distorsionar y el resto con una distorsión del 1% al 30%.
Se usará una **distribución uniforme** para ubicar el 90% de ejemplos en el rango de distorsión entre 0.01 y 0.30.

Cada dataset será un `pd.DataFrame` de `pandas` que contiene columnas del `0` al `99` (una por cada celda de la matriz) y una columna final `class` que indica la clase del patrón ('b', 'd' o 'f').


In [3]:
def generate_dataset(n_samples: int) -> pd.DataFrame:
    """
    Generates a dataset of pattern samples.
    The 90% of samples will have noise between 0.01 and 0.30.

    Args:
        n_samples: Number of samples to generate.
    Returns:
        A dataframe with 100 columns for the flattened pattern and 1 column 'class' for the pattern class.
    """
    columns = [str(i) for i in range(100)] + ["class"]
    df = pd.DataFrame(0, index=np.arange(n_samples), columns=columns)
    df = df.astype({"class": "str"})

    for i in range(n_samples):
        # 10% without distortion, 90% with distortion between 1% and 30%
        if i < int(0.1 * n_samples):
            noise = 0.0
        else:
            noise = RNG.uniform(0.01, 0.30)

        # Pick a pattern at random
        pattern = RNG.choice(list(PATTERNS.keys()))

        sample = generate_sample(pattern, noise).flatten()
        df.iloc[i, :100] = sample
        df.loc[i, "class"] = pattern

    return df


df_100 = generate_dataset(100)
df_500 = generate_dataset(500)
df_1000 = generate_dataset(1000)

print("Dataset de 1000 ejemplos:")
print(df_1000)

Dataset de 1000 ejemplos:
     0  1  2  3  4  5  6  7  8  9  ...  91  92  93  94  95  96  97  98  99  class
0    0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      d
1    0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      f
2    0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      f
3    0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      b
4    0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      b
..  .. .. .. .. .. .. .. .. .. ..  ...  ..  ..  ..  ..  ..  ..  ..  ..  ..    ...
995  0  0  0  1  0  1  0  0  0  0  ...   0   0   0   0   0   1   0   0   0      b
996  0  0  0  1  0  0  0  0  0  0  ...   0   0   0   0   0   0   1   0   1      f
997  0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      b
998  0  1  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      b
999  1  0  0  0  0  1  1  0  0  1  ...   0   1   0   0   0   1   1   1  

Por cada dataset deberán construirse tres conjuntos de validación con 10%, 20% y 30% de los ejemplos.
Esto da un total de 9 pares de datasets de entrenamiento y datasets de validación:

1. Dataset original de 100 ejemplos con 90% para entrenamiento y 10% para validación.
2. Dataset original de 100 ejemplos con 80% para entrenamiento y 20% para validación.
3. Dataset original de 100 ejemplos con 70% para entrenamiento y 30% para validación.
4. Dataset original de 500 ejemplos con 90% para entrenamiento y 10% para validación.
5. Dataset original de 500 ejemplos con 80% para entrenamiento y 20% para validación.
6. Dataset original de 500 ejemplos con 70% para entrenamiento y 30% para validación.
7. Dataset original de 1000 ejemplos con 90% para entrenamiento y 10% para validación.
8. Dataset original de 1000 ejemplos con 80% para entrenamiento y 20% para validación.
9. Dataset original de 1000 ejemplos con 70% para entrenamiento y 30% para validación.

In [ ]:
VALIDATION_RATIOS = [0.1, 0.2, 0.3]


def split_dataset(df: pd.DataFrame, validation_ratio: float):
    """
    Splits the dataset into training and validation sets.

    Args:
        df: The dataset to split.
        validation_ratio: Proportion (between 0 and 1) of the dataset to include in the validation set.
    Returns:
        tuple: (training_df, validation_df).
    """
    shuffled_df = df.sample(frac=1, random_state=SEED).reset_index(drop=True)
    validation_size = int(len(df) * validation_ratio)
    validation_df = shuffled_df.iloc[:validation_size].reset_index(drop=True)
    training_df = shuffled_df.iloc[validation_size:].reset_index(drop=True)
    return training_df, validation_df


# Store all datasets in one dictionary
datasets = {}
for df_name, df in [("100", df_100), ("500", df_500), ("1000", df_1000)]:
    datasets[df_name] = {}
    for validation_ratio in VALIDATION_RATIOS:
        training, validation = split_dataset(df, validation_ratio)
        key = f"val_{int(validation_ratio * 100)}%"
        # Store training dataset and validation dataset for each dataset, for each validation split
        datasets[df_name][key] = {
            "training": training,
            "validation": validation,
        }

## 🧠 Implementación del Modelo

Se debe implementar el algoritmo MLP que permita, dado un dataset, parametrizar la cantidad de capas, neuronas y funciones de activación con los que se entrenará la red neuronal.

Requerimientos para la arquitectura del modelo:

- 1 o 2 capas ocultas.
- De 5 a 10 neuronas por capa.
- Funciones de activación: lineal y sigmoidal.
- Coeficiente de aprendizaje entre 0 y 1.
- Término momento entre 0 y 1.

Para estudiar el algoritmo se utiliza de referencia la bibliografía recomendada por la cátedra [disponible en el campus virtual](https://frre.cvg.utn.edu.ar/pluginfile.php/105673/mod_label/intro/Perceprtr%C3%B3n-MLP.pdf):

- Hilera, J. R. Martinez, V. J. (2000) Redes Neuronales Artificiales. Fundamentos, modelos y aplicaciones. Alfaomega.

También se consulta material complementario para resolver dudas durante la implementación:

- Prince, S. J. D. (2023). Understanding deep learning. The MIT Press. [http://udlbook.com](http://udlbook.com).

### Arquitectura

La arquitectura de esta red neuronal feedforward (MLP) consiste en:

- **Capa de entrada:** 100 neuronas, una por cada celda de la matriz.
- **Capa oculta 1:** 10 neuronas.
- **Capa oculta 2:** 5 neuronas.
- **Capa de salida:** 3 neuronas, una para la clasificación de cada patrón 'b', 'd', 'f'.

Próximamente los hiperparámetros serán parametrizables, pero se utiliza esta arquitectura como base.

![Diagrama de la arquitectura del MLP](assets/architecture.png)

Esta red neuronal será una función $f[X, \phi] = Y$ que clasifica el contenido de una matriz 10x10 en uno de tres patrones. 
El contenido de la matriz 10x10 se representa como el vector de entrada $X = [x_0, x_1, \dots, x_{99}]^T$.
El vector de salida $Y = [y_b, y_d, y_f]^T$ contiene la predicción del modelo para cada patrón.
El conjunto de parámetros $\phi = \set{B_k, W_k}_{k=0}^K$ del modelo contiene:

- El vector de _biases_ $B_k$ que contribuyen a la capa $k+1$. Es de tamaño $D_{k+1}$.
- Los _weights_ (pesos) $W_k$ que son aplicados a la capa $k$ y que contribuyen a la capa $k+1$. Es de tamaño $D_{k+1}  \times D_k$.

La red neuronal se puede representar utilizando notación matricial:

$$
\begin{aligned}
X &= [x_0, x_2, \dots, x_{99}]^T \\
H_1 &= a \left[ B_{0} + W_{0} X \right] \\
H_2 &= a \left[ B_{1} + W_{1} H_1 \right] \\
Y &= B_2 + W_{2} H_2 \\
\end{aligned}
$$

donde $a[\bullet]$ es una función que aplica la función de activación por separado a cada elemento de su vector de entrada (Prince, 2023).

Esa notación se puede expandir de la siguiente manera:

$$\begin{aligned}
H_1
&=
\begin{bmatrix}
h_0 \\
h_1 \\
\vdots \\
h_9
\end{bmatrix}
= 
a
\left[
\begin{bmatrix}
b_{0} \\
b_{1} \\
\vdots \\
b_{9}
\end{bmatrix}
+
\begin{bmatrix}
w_{01} & w_{02} & \dots & w_{0\ 99} \\
w_{11} & w_{12} & \dots & w_{1\ 99} \\
\vdots & \vdots & \ddots & \vdots \\
w_{91} & w_{92} & \dots & w_{9\ 99} \\
\end{bmatrix}
\begin{bmatrix}
x_{0} \\
x_{1} \\
\vdots \\
x_{99}
\end{bmatrix}
\right]  \\

H_2
&=
\begin{bmatrix}
h'_0 \\
h'_1 \\
\vdots \\
h'_4
\end{bmatrix}
= 
a
\left[
\begin{bmatrix}
b'_{0} \\
b'_{1} \\
\vdots \\
b'_{4}
\end{bmatrix}
+
\begin{bmatrix}
w'_{01} & w'_{02} & \dots & w'_{09} \\
w'_{11} & w'_{12} & \dots & w'_{19} \\
\vdots & \vdots & \ddots & \vdots \\
w'_{41} & w'_{42} & \dots & w'_{49} \\
\end{bmatrix}
\begin{bmatrix}
h_{0} \\
h_{1} \\
\vdots \\
h_{9}
\end{bmatrix}
\right]  \\

Y
&=
\begin{bmatrix}
y_b \\
y_d \\
y_f
\end{bmatrix}
= 
\begin{bmatrix}
b''_{0} \\
b''_{1} \\
b''_{2}
\end{bmatrix}
+
\begin{bmatrix}
w''_{01} & w''_{02} & \dots & w''_{04} \\
w''_{11} & w''_{12} & \dots & w''_{14} \\
w''_{21} & w''_{22} & \dots & w''_{24} \\
\end{bmatrix}
\begin{bmatrix}
h'_{0} \\
h'_{1} \\
\vdots \\
h'_{4}
\end{bmatrix} \\
\end{aligned}
$$

Por ejemplo, la segunda neurona de la primera capa oculta se calcula como $h_1 = a[b_1 + w_{11}x_0 + \dots + w_{1\ 99}x_{99}]$.

Los parámetros de $\phi$ serán inicializados utilizando la técnica de Xavier Glorot, y el entrenamiento del MLP consistirá en ajustar estos parámetros hasta lograr resultados aceptables.

### Funciones de Activación

Si bien existen múltiples funciones de activación, el trabajo práctico requiere dos en particular:

- Lineal.
- Sigmoidal.

![Funciones de activación](assets/activation_functions.png)


### Regla de Aprendizaje

Se debe elegir la regla de aprendizaje o _loss function_ a utilizar.
Hilera & Martinez presentan la regla _Least Mean Squared_ o _regla delta_:

$$
\epsilon_k^2 = \frac{1}{2L} \sum_{k=1}^L (d_k - s_k)^2
$$


mientras que Prince presenta una función _Least Squares_ muy similar que omite el coeficiente constante:

$$
L[\phi] = \sum_{i=1}^I (y_i - f[X_i, \phi])^2
$$

Ambas consisten en comparar la salida obtenida contra la deseada para obtener el costo o pérdida.
El costo se debería reducir en cada iteración del entrenamiento del MLP a medida que se ajustan los parámetros.


### Momento

El _momento_ es una modificación al algoritmo _backpropagation_ del gradiente descendiente para suavizar el progreso del algoritmo y evitar oscilaciones.
Matemáticamente es un coeficiente $\beta$ que se agrega para considerar el valor de la iteración anterior de un parámetro al momento de calcular su nuevo valor en la iteración siguiente.

Hilera & Martinez lo presentan como:

$$
w_{ji} (t+1) = w_{ji}(t) \alpha \delta_{pj}y_{pi} + \underbrace {\beta (w_{ji}(t) - w_{ji} (t-1))}_\text{Término momento} \\
$$

mientras que Prince lo presenta como:

$$
\begin{aligned}
\mathbf{m}_{t+1} &\leftarrow \beta \cdot \mathbf{m}_t 
  + (1 - \beta) \sum_{i \in \mathcal{B}_t} 
  \frac{\partial \ell_i[\boldsymbol{\phi}_t]}{\partial \boldsymbol{\phi}}, \\[6pt]
\boldsymbol{\phi}_{t+1} &\leftarrow 
  \boldsymbol{\phi}_t - \alpha \cdot \mathbf{m}_{t+1},
\end{aligned}
$$

In [ ]:
SIZE_INPUTS = 100
SIZE_H1 = 10
SIZE_H2 = 5
SIZE_OUTPUTS = 3


class MLP:
    """
    Multilayer Perceptron implementation for pattern classification.
    """

    def __init__(
        self,
        architecture: List[int] = [100, 10, 5, 3],
        learning_rate: float = 0.1,
        momentum: float = 0.0,
        activation_type: Literal["sigmoid", "linear"] = "sigmoid",
    ):
        """
        Initialize the MLP with given hyperparameters.

        Args:
            architecture: List with the size of each layer [input, hidden1, hidden2, output].
            learning_rate: Learning rate for gradient descent (0 < lr <= 1).
            momentum: Momentum coefficient for parameter updates (0 <= momentum < 1).
            activation_type: Activation function type.
        """
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.activation_type = activation_type

        self.architecture = architecture
        self.input_size = architecture[0]
        self.hidden1_size = architecture[1]
        self.hidden2_size = architecture[2]
        self.output_size = architecture[3]

        self._initialize_parameters()
        self._initialize_momentum()

    def _initialize_parameters(self):
        """
        Initialize weights and biases using Xavier initialization, where each
        parameter will be drawn from a normal distribution with mean of `0` and
        a standard deviation of `sqrt(2 / (input_layer_size + output_layer_size))`.
        This helps prevent vanishing or exploding gradients.
        """
        # Initialize layer 1 parameters
        std1 = np.sqrt(2.0 / (self.input_size + self.hidden1_size))
        self.W1 = RNG.normal(0, std1, (self.hidden1_size, self.input_size))
        self.b1 = np.zeros(self.hidden1_size)

        # Initialize layer 2 parameters
        std2 = np.sqrt(2.0 / (self.hidden1_size + self.hidden2_size))
        self.W2 = RNG.normal(0, std2, (self.hidden2_size, self.hidden1_size))
        self.b2 = np.zeros(self.hidden2_size)

        # Initialize output layer parameters
        std3 = np.sqrt(2.0 / (self.hidden2_size + self.output_size))
        self.W3 = RNG.normal(0, std3, (self.output_size, self.hidden2_size))
        self.b3 = np.zeros(self.output_size)

        print("Parameters initialized:")
        print(f"  W1 shape: {self.W1.shape}, b1 shape: {self.b1.shape}")
        print(f"  W2 shape: {self.W2.shape}, b2 shape: {self.b2.shape}")
        print(f"  W3 shape: {self.W3.shape}, b3 shape: {self.b3.shape}")
        print(
            f"  Total parameters: {self.W1.size + self.b1.size + self.W2.size + self.b2.size + self.W3.size + self.b3.size}"
        )

    def _initialize_momentum(self):
        """
        Initialize momentum terms for parameter updates.
        These store the previous update directions to add momentum.
        """
        self.dW1_prev = np.zeros_like(self.W1)
        self.db1_prev = np.zeros_like(self.b1)
        self.dW2_prev = np.zeros_like(self.W2)
        self.db2_prev = np.zeros_like(self.b2)
        self.dW3_prev = np.zeros_like(self.W3)
        self.db3_prev = np.zeros_like(self.b3)

    def get_activation_function(self):
        """Return the activation function based on the activation type."""
        if self.activation_type == "sigmoid":
            return lambda x: 1 / (1 + np.exp(-x))
        elif self.activation_type == "linear":
            return lambda x: x

    def feedforward(self, X: np.ndarray) -> np.ndarray:
        """
        Perform forward propagation through the network.

        Args:
            X: Array of 100 values (0 or 1) representing the input data from a sample.
        Returns:
            Array of 3 items with the predictions for classes 'b', 'd' and 'f'.
        """
        a = self.get_activation_function()

        # Forward pass from input layer to hidden layer 1
        h1 = a(self.b1 + np.dot(self.W1, X.T).T)

        # Forward pass from hidden layer 1 to hidden layer 2
        h2 = a(self.b2 + np.dot(self.W2, h1.T).T)

        # Forward pass from hidden layer 2 to output layer
        y = self.b3 + np.dot(self.W3, h2.T).T

        return y


mlp = MLP()

Parameters initialized:
  W1 shape: (10, 100), b1 shape: (10,)
  W2 shape: (5, 10), b2 shape: (5,)
  W3 shape: (3, 5), b3 shape: (3,)
  Total parameters: 1083


In [6]:
print("Testing feedforward with sample data...")

# Test with a single sample
sample_pattern = generate_sample("b", 0.0)
print("Input pattern:")
print_sample(sample_pattern)

output_single = mlp.feedforward(sample_pattern)
print("Single sample output:")
print(output_single)

# Test with multiple samples
multiple_patterns = np.array(
    [
        generate_sample("b", 0.0),
        generate_sample("d", 0.0),
        generate_sample("f", 0.0),
    ]
)
output_multiple = mlp.feedforward(multiple_patterns)
print("Multiple samples output:")
print(output_multiple)

Testing feedforward with sample data...
Input pattern:
0 0 0 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0
0 0 1 1 1 1 1 0 0 0
0 0 1 0 0 0 0 1 0 0
0 0 1 0 0 0 0 1 0 0
0 0 1 0 0 0 0 1 0 0
0 0 1 1 1 1 1 0 0 0
0 0 0 0 0 0 0 0 0 0
Single sample output:
[ 0.71048979 -0.3419544  -0.58371697]
Multiple samples output:
[[ 0.71048979 -0.3419544  -0.58371697]
 [ 0.69937229 -0.34897471 -0.57442121]
 [ 0.70946331 -0.37255956 -0.58141278]]
